In [27]:
import torch
import numpy as np

In [28]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [29]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取⼩批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [30]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-1.5030, -0.1549],
        [-2.3286, -0.1789],
        [-0.1186, -0.6824],
        [ 1.3233, -0.4252],
        [-0.1175,  0.9617],
        [-1.4057,  0.6514],
        [ 1.6332, -0.2635],
        [-0.4076, -0.1192],
        [ 1.4033,  0.9572],
        [-1.7858, -0.3903]]) tensor([ 1.7216,  0.1302,  6.2857,  8.2927,  0.6973, -0.8300,  8.3562,  3.8012,
         3.7494,  1.9479])


In [31]:
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使⽤print可以打印出⽹络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [42]:
# 写法⼀
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传⼊其他层
)

# 写法⼆
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    # ('linear1', nn.Linear(num_inputs, 1))
]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [43]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.6358,  0.1562]], requires_grad=True)
Parameter containing:
tensor([0.6829], requires_grad=True)


In [44]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) # 也可以直接修改bias的data:net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [45]:
loss = nn.MSELoss()

In [46]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [47]:
optimizer = optim.SGD([
                # 如果对某个参数不指定学习率，就使⽤最外层的默认学习率
                {'params': net.linear.parameters()} # lr=0.03
                # {'params': net.linear1.parameters(), 'lr':0.01}   # linear1是上面自己定义的网络名字
            ], lr=0.03)

In [48]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

In [50]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()  # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 2.329097
epoch 2, loss: 0.883349
epoch 3, loss: 0.376752


In [51]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.7381, -3.0980]], requires_grad=True)
4.2 Parameter containing:
tensor([3.7993], requires_grad=True)
